In [ ]:
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot
import datetime as dt

In [ ]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [ ]:
df_raw.shape

In [ ]:
df_an = df_raw.copy() #df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [ ]:
df_an.shape

In [ ]:
df_an = RSI(df_an)

In [ ]:
df_an.tail()

In [ ]:
df_an = apply_patterns(df_an)

In [ ]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [ ]:
df_an.columns

In [ ]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14']

In [ ]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [ ]:
df_slim.head()

In [ ]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE

In [ ]:
df_slim['SIGNAL'] = df_slim.apply(apply_signal, axis=1)

In [ ]:
df_slim['SIGNAL'].value_counts()

In [ ]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * PROFIT_FACTOR + row.mid_c
    else:
        return 0.0

def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [ ]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [ ]:
df_slim[df_slim.SIGNAL==BUY].head()

In [ ]:
df_plot = df_slim.iloc[0:30]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker = dict(color=marker_colors[i], size=12)
    ))
    
cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'])

In [ ]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c 
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
    
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
    
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)

In [ ]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))


In [ ]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [ ]:
df_results.result.sum()

In [ ]:
df_results.sort_values(by="start_index", inplace=True)

In [ ]:
df_results

In [ ]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [ ]:
df_raw.time.max()

In [1]:
from dateutil import parser

In [ ]:
time_min = parser.parse("2021-12-15T10:00:00Z")
time_max = parser.parse("2021-12-15T11:00:00Z")
df_m5_s = df_m5[(df_m5.time>=time_min)&(df_m5.time<=time_max)]
df_raw_s = df_raw[(df_raw.time>=time_min)&(df_raw.time<=time_max)]

In [ ]:
df_m5_slim = df_m5[['time', 'mid_h', 'mid_l']].copy()

In [ ]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy()

In [ ]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [ ]:
df_signals['start_index_h1'] = df_signals.index

In [ ]:
df_signals.head()

In [ ]:
df_signals.columns

In [ ]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l','ask_c','bid_c',
                 'ENGULFING','EMA_200', 'RSI_14', 'direction'], 
                axis=1, inplace=True)

In [ ]:
df_signals.head()

In [ ]:
df_signals.rename(columns={
    'mid_c': 'start_price',
    'm5_start': 'time'
}, inplace=True)

In [ ]:
df_signals.head()

In [ ]:
df_m5_slim.head(2)

In [ ]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')

In [ ]:
merged[merged.SIGNAL.isna() == False]
merged.fillna(0, inplace=True)

In [ ]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [ ]:
merged

In [ ]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1
    
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
    
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)

In [ ]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():
    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))
        
    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]

In [ ]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [ ]:
df_res_m5.result.sum()